In [1]:
import keras as keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Bidirectional
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers
from keras.layers import Input, LSTM, Dense, Concatenate
from keras.models import Model
from itertools import permutations
import keras.backend as K
import pandas as pd
import numpy as np

2023-12-09 13:59:55.346234: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 13:59:55.346482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 13:59:55.397205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 13:59:55.576618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 13:59:56.884119: W tensorflow/compiler/tf2

In [2]:
def permutation_invariant_loss(y_true, y_pred):
    # Sort the true and predicted values along the last axis
    y_true_sorted = tf.sort(y_true, axis=-1)
    y_pred_sorted = tf.sort(y_pred, axis=-1)

    # Calculate the mean absolute error between the sorted true and predicted values
    return K.mean(K.abs(y_true_sorted - y_pred_sorted), axis=-1)


In [14]:
csv_file_path = 'sinusoid_dataset.csv'
df = pd.read_csv(csv_file_path)
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split()])).values
labels = df['Label'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
print(features.shape)
print(labels.shape)

(100000,)
(100000,)


In [15]:
padded_array = pad_sequences(labels, padding='post', maxlen=3)
print(padded_array.shape)
labels = np.vstack(padded_array)
features = np.vstack(features)
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

(100000, 3)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train_categorical shape:", y_train.shape)
print("y_test_categorical shape:", y_test.shape)

X_train shape: (80000, 299)
X_test shape: (20000, 299)
y_train_categorical shape: (80000, 3)
y_test_categorical shape: (20000, 3)


In [17]:
# Define three LSTM branches
inputs = keras.Input(shape=(299,1))
# Branch 1
lstm_branch_1 = LSTM(units=128, return_sequences=True)(inputs)
flatten_1 = layers.Flatten()(lstm_branch_1)
output_branch_1 = Dense(units=1, activation='linear', name='output_branch_1')(flatten_1)

# Branch 2
lstm_branch_2 = LSTM(units=128, return_sequences=True)(lstm_branch_1)  # Pass the output of branch 1
flatten_2 = layers.Flatten()(lstm_branch_2)
output_branch_2 = Dense(units=1, activation='linear', name='output_branch_2')(flatten_2)

# Branch 3
lstm_branch_3 = LSTM(units=128, return_sequences=True)(lstm_branch_2)  # Pass the output of branch 2
flatten_3 = layers.Flatten()(lstm_branch_3)
output_branch_3 = Dense(units=1, activation='linear', name='output_branch_3')(flatten_3)

# Concatenate the outputs of the three branches
merged_output = Concatenate(axis=-1)([output_branch_1, output_branch_2, output_branch_3])

# Define the model
model = Model(inputs=inputs, outputs=merged_output)
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 1)]             0         []                            
                                                                                                  
 lstm_6 (LSTM)               (None, 299, 128)             66560     ['input_3[0][0]']             
                                                                                                  
 lstm_7 (LSTM)               (None, 299, 128)             131584    ['lstm_6[0][0]']              
                                                                                                  
 lstm_8 (LSTM)               (None, 299, 128)             131584    ['lstm_7[0][0]']              
                                                                                            

In [18]:
model.compile(optimizer='adam',
              loss=permutation_invariant_loss, 
              metrics=['mae'])

model.fit(X_train_tensor, y_train_tensor, epochs=10, batch_size=256, validation_split=0.2)

test_loss, test_accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')

Epoch 1/10
250/250 [==============================] - 35s 126ms/step - loss: 8.5076 - mae: 8.5078 - val_loss: 8.2060 - val_mae: 8.2060
Epoch 2/10
250/250 [==============================] - 31s 125ms/step - loss: 7.7501 - mae: 7.7522 - val_loss: 7.8850 - val_mae: 7.8850
Epoch 3/10
250/250 [==============================] - 31s 122ms/step - loss: 5.9000 - mae: 5.9006 - val_loss: 2.5514 - val_mae: 2.5517
Epoch 4/10
250/250 [==============================] - 31s 125ms/step - loss: 1.2869 - mae: 1.2873 - val_loss: 0.3161 - val_mae: 0.3168
Epoch 5/10
250/250 [==============================] - 32s 129ms/step - loss: 0.4092 - mae: 0.4094 - val_loss: 0.2627 - val_mae: 0.2627
Epoch 6/10
250/250 [==============================] - 31s 123ms/step - loss: 0.2578 - mae: 0.2579 - val_loss: 0.3200 - val_mae: 0.3201
Epoch 7/10
250/250 [==============================] - 31s 125ms/step - loss: 0.2024 - mae: 0.2025 - val_loss: 0.2548 - val_mae: 0.2548
Epoch 8/10
250/250 [==============================] - 3

In [20]:
csv_file_path = 'samplesreal.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract features and labels
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
labels = [[label] for label in df['Label'].tolist()]
# Convert the string of up to 5 numbers to a NumPy array of floats
padded_array = pad_sequences(labels, padding='post', maxlen=3)
labels = np.vstack(padded_array)
features = np.vstack(features)
print(labels.shape)
print(features.shape)

#means = np.mean(features, axis=1, keepdims=True)
#print(means)
#features = features - means
scaler = MinMaxScaler()
features = scaler.fit_transform(features)
print(features)
print(labels)
#change to 50
#features = features * 1
print("Real Sinusoid Tests:\n")
for i in range(8):
  print(labels[i:i+1])
  print(model.predict(features[i:i+1]))

print("Synthetic Sinusoid Tests:\n")
for i in range(100):
  print(y_test[i:i+1])
  print(model.predict(X_test[i:i+1]))

(8, 3)
(8, 299)
[[0.28537688 0.18229077 0.17790109 ... 0.         0.         0.0307157 ]
 [0.79716991 0.44791631 0.35580451 ... 0.66950575 0.72222207 0.51535891]
 [0.41037798 0.46527826 0.68164823 ... 0.05962498 0.23931506 0.40273043]
 ...
 [1.         1.         0.90636699 ... 0.11073271 0.72649632 1.        ]
 [0.98113174 0.08159836 0.         ... 1.         1.         0.27644981]
 [0.54245284 0.06076359 1.         ... 0.79897683 0.97008562 0.06143353]]
[[ 1  0  0]
 [10  0  0]
 [19  0  0]
 [27  0  0]
 [36  0  0]
 [45  0  0]
 [54  0  0]
 [90  0  0]]
Real Sinusoid Tests:

[[1 0 0]]
1/1 [==============================] - 0s 53ms/step
[[ 5.1917028e+00 -7.9683699e-03 -6.8170216e-04]]
[[10  0  0]]
1/1 [==============================] - 0s 56ms/step
[[-6.2845087e+00  3.4585493e-03  2.0320415e-03]]
[[19  0  0]]
1/1 [==============================] - 0s 95ms/step
[[ 7.0535731e+00 -9.2867477e-04  4.2381353e-04]]
[[27  0  0]]
1/1 [==============================] - 0s 43ms/step
[[ 1.7795124e+01 